In [1]:
#Initialize
from factfinder.calculate import Calculate
from factfinder.median import Median
from factfinder.utils import get_median
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

# pd.options.display.float_format = "{:,.18f}".format

try:
    env_path = "../.env"
    load_dotenv(dotenv_path=env_path)
except:
    print(".env file is missing ...")

calculate = Calculate(api_key=os.environ["API_KEY"], year=2019, source="acs", geography='2010_to_2020')

In [2]:
##INPUTS -- change here
pff_variable = 'mdvl'
geotype = 'CT20'
census_geoid_list = ['36005005002']

In [3]:
# See all digits of ratio
ratio = calculate.geo.ratio
ratio.loc[ratio.geoid_ct2020.isin(census_geoid_list), :]


,geoid_ct2010,geoid_ct2020,ratio
30,36005005002,36005005002,1.0


In [4]:
# Get ranges and design factor from metadata
ranges = calculate.meta.median_ranges(pff_variable)
design_factor = calculate.meta.median_design_factor(pff_variable)
print(f"\nDesign factor: {design_factor}")


Design factor: 1.4


In [5]:
# Calculate inputs in 2020 geogs
df = calculate.calculate_e_m_multiprocessing(list(ranges.keys()), geotype)

In [6]:
# 3. create a pivot table with census_geoid as the index, and
# pff_variable as column names. df_pivoted.e -> the estimation dataframe
df_pivoted = df.loc[df.census_geoid.isin(census_geoid_list), ["census_geoid", "pff_variable", "e"]].pivot(
    index="census_geoid", columns="pff_variable", values=["e"]
)
df_pivoted = df_pivoted.round(16)
df_pivoted

e                                                    \
pff_variable ov100t124 ov125t149 ov150t174 ov150t199m ov175t199 ov1t149m   
census_geoid                                                               
36005005002        0.0       0.0       0.0        0.0       0.0      0.0   

                                                     ...                    \
pff_variable ov200t249 ov250t299 ov2milpl ov300t399  ... ovl25t29 ovl30t34   
census_geoid                                         ...                     
36005005002        0.0       0.0     12.0       0.0  ...      0.0      0.0   

                                                                             \
pff_variable ovl35t39 ovl40t49 ovl50t59 ovl60t69 ovl70t79 ovl80t89 ovl90t99   
census_geoid                                                                  
36005005002       0.0      0.0      0.0      0.0      0.0      0.0      0.0   

                     
pff_variable ovlu10  
census_geoid         
36005005002     0.0  

[1 rows x 26 columns]

In [7]:
# Empty dataframe to store the results
results = pd.DataFrame()
results["census_geoid"] = df_pivoted.index
results["pff_variable"] = pff_variable
results["geotype"] = geotype
results

,census_geoid,pff_variable,geotype
0,36005005002,mdvl,CT20


In [8]:
# 4. calculate median estimation using get_median
results["e"] = (
    df_pivoted.e.loc[
        df_pivoted.e.index == results.census_geoid, list(ranges.keys())
    ]
    .apply(lambda row: Median(ranges, row, design_factor).median, axis=1)
    .to_list()
)

DEBUG:root:N/2 is in range [400000, 499999]
DEBUG:root:
C_19: Cumulative frequency up to bin below N/2: 0.0
L_20: Lower boundary of median group: 400000
F_20: Frequency within median group: 51.0
W_20: Width of median group: 99999
DEBUG:root:
MEDIAN: 497057.8529411765



In [9]:
# 5. Calculate median moe using get_median_moe
# Note that median moe calculation needs the median estimation
# so we seperated df_pivoted.m out as a seperate dataframe
e = df_pivoted.e.copy()
e["e"] = results.loc[e.index == results.census_geoid, "e"].to_list()
results["m"] = (
    e.loc[e.index == results.census_geoid, list(ranges.keys()) + ["e"]]
    .apply(lambda row: Median(ranges, row, design_factor).median_moe, axis=1)
    .to_list()
)

DEBUG:root:N/2 is in range [400000, 499999]
DEBUG:root:
C_19: Cumulative frequency up to bin below N/2: 0.0
L_20: Lower boundary of median group: 400000
F_20: Frequency within median group: 51.0
W_20: Width of median group: 99999
DEBUG:root:
MEDIAN: 497057.8529411765

DEBUG:root:
            UPPER_BOUND:
            -----
            A1=500000, A2=750000, C1=51.515151515151516, C2=87.87878787878788
            
DEBUG:root:lower_bin not in bottom bin and is the first none-zero bin
DEBUG:root:
            LOWER_BOUND:
            -----
            A1=0, A2=10000, C1=0.0, C2=51.515151515151516
            
DEBUG:root:N/2 is in range [400000, 499999]
DEBUG:root:
C_19: Cumulative frequency up to bin below N/2: 0.0
L_20: Lower boundary of median group: 400000
F_20: Frequency within median group: 51.0
W_20: Width of median group: 99999
DEBUG:root:
MEDIAN: 497057.8529411765

DEBUG:root:
MEDIAN STATS:
-----
Median = 497057.8529411765
Median_MOE = 543797.9570562358
B = 99.0
se_50 = 25.6432399976

In [10]:
results.head()

,census_geoid,pff_variable,geotype,e,m
0,36005005002,mdvl,CT20,497057.852941,543797.957056


In [11]:
# Peform full calculation (including cleaning/rounding) to show display output
import logging
logging.basicConfig(level="ERROR")
full_calc = calculate(pff_variable, geotype)
full_calc.loc[full_calc.census_geoid.isin(census_geoid_list),:]

,census_geoid,labs_geoid,geotype,labs_geotype,pff_variable,c,e,m,p,z
30,36005005002,2005002,CT20,CT2020,mdvl,66.5,497058.0,543798.0,NaN,NaN
